In [1]:
import json
import sys
import hvplot.pandas
import pandas as pd
from utils import load_config, fetch_api_data, write_to_csv

In [2]:
# Load config file
config = load_config('config.json')

# Get API key
weather_api_key = config.get('National Centers for Environmental Information', {}).get('key')

# Set base URL
base_url = f"https://www.ncei.noaa.gov/cdo-web/api/v2"

# Set endpoints
endpoints = ["datasets", "datacategories", "datatypes", "locationcategories", "locations", "stations", "data"]

# Check if key exists in config file 
if not weather_api_key:
    print("Weather API key not found in the configuration file.")
    sys.exit()

In [3]:
# 'nx3tvs'       - (Point)   NEXRAD Level-3 Tornado Vortex Signatures
# 'nx3meso'      - (Point)   NEXRAD Level-3 Mesocyclone Signatures
# 'nx3hail'      - (Point)   NEXRAD Level-3 Hail Signatures
# 'nx3structure' - (Point)   NEXRAD Level-3 Storm Cell Structure Information
# 'warn'         - (Polygon) Severe Thunderstorm, Tornado, Flash Flood and Special Marine warnings
datasets = ["nx3tvs"]
outputFormat = "json"
daterange = "20240730:20240731"  # "periodOfRecord"

for dataset in datasets:
    base_url = f"https://www.ncdc.noaa.gov/swdiws/{outputFormat}/{dataset}/{daterange}"
    filename = f"swdiws_{dataset}.csv"

    data = fetch_api_data(base_url)
    #print(json.dumps(data, indent=2))
    #print(data)

    if data and "result" in data:
        write_to_csv(data["result"], filename, "w")
    else:
        print(f"No data found or invalid response format for dataset: {dataset}")


In [15]:
# 'nx3tvs'       - (Point)   NEXRAD Level-3 Tornado Vortex Signatures
# 'nx3meso'      - (Point)   NEXRAD Level-3 Mesocyclone Signatures
# 'nx3hail'      - (Point)   NEXRAD Level-3 Hail Signatures
# 'nx3structure' - (Point)   NEXRAD Level-3 Storm Cell Structure Information
# 'warn'         - (Polygon) Severe Thunderstorm, Tornado, Flash Flood and Special Marine warnings
datasets = ["nx3tvs"]
outputFormat = "geojson"
daterange = "20060505:20060506"  # "periodOfRecord"
numResults = 250

# Initialize an empty list to store merged data
merged_data_list = []

for dataset in datasets:
    base_url = f"https://www.ncdc.noaa.gov/swdiws/{outputFormat}/{dataset}/{daterange}/{numResults}"
    filename = f"swdiws_{dataset}_{outputFormat}.csv"

    data = fetch_api_data(base_url)
    # Data is nested, retrieve "features" dictionary
    rows = data["features"]
    
    # Iterate over each record in rows as that is nested as well
    for record in rows:
       # Merge the 'properties' and 'geometry' dictionaries
        merged_data = {**record["properties"], **record["geometry"]}
        # Append the merged data to the list
        merged_data_list.append(merged_data)

    # Convert to DataFrame
    merged_df = pd.DataFrame(merged_data_list)

# Split the coordinates column into latitude and longitude
merged_df[['longitude', 'latitude']] = pd.DataFrame(merged_df['coordinates'].tolist(), index=merged_df.index)
# Drop the original coordinates column
merged_df = merged_df.drop(columns=['coordinates'])
merged_df.head(25)

    # if data and "features" in data:
    #     write_to_csv(data["features"], filename, "w")
    # else:
    #     print(f"No data found or invalid response format for dataset: {dataset}")

,CELL_TYPE,MAX_SHEAR,WSR_ID,MXDV,CELL_ID,ZTIME,AZIMUTH,RANGE,type,longitude,latitude
0,TVS,403,KBMX,116,Q0,2006-05-05T00:05:50Z,217,7,Point,-86.8536,33.0786
1,TVS,421,KBMX,120,Q0,2006-05-05T00:10:02Z,208,5,Point,-86.8166,33.0983
2,TVS,17,KSJT,52,P2,2006-05-05T00:12:34Z,106,49,Point,-99.5771,31.1422
3,TVS,25,KSJT,62,B4,2006-05-05T00:17:31Z,297,40,Point,-101.1882,31.6724
4,TVS,34,KMAF,111,H4,2006-05-05T00:29:13Z,333,53,Point,-102.6644,32.7307
5,TVS,24,KLBB,78,N0,2006-05-05T00:31:25Z,241,51,Point,-102.7005,33.2380
6,TVS,46,KMAF,145,H4,2006-05-05T00:33:25Z,334,52,Point,-102.6394,32.7227
7,TVS,34,KMAF,107,H4,2006-05-05T00:37:37Z,334,50,Point,-102.6219,32.6927
8,TVS,29,KMAF,91,H4,2006-05-05T00:41:51Z,335,51,Point,-102.6148,32.7141
9,TVS,35,KLBB,100,N0,2006-05-05T00:44:33Z,245,46,Point,-102.6434,33.3266


In [16]:
# Convert 'MXDV' to numeric, forcing errors to NaN if conversion fails
merged_df['MXDV'] = pd.to_numeric(merged_df['MXDV'], errors='coerce')
# Ensure 'WSR_ID' is treated as a string
merged_df['WSR_ID'] = merged_df['WSR_ID'].astype(str)
print(merged_df.dtypes)

CELL_TYPE     object
MAX_SHEAR     object
WSR_ID        object
MXDV           int64
CELL_ID       object
ZTIME         object
AZIMUTH       object
RANGE         object
type          object
longitude    float64
latitude     float64
dtype: object


In [17]:
%%capture --no-display
# Configure the map plot
tornadoes_plot = merged_df.hvplot.points(
    "longitude",
    "latitude",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="MXDV",
    color="WSR_ID"
)

# Display the map
tornadoes_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (WSR_ID,MXDV)

In [5]:
# Define headers
headers = {"token": weather_api_key}
# Define HTTPS call timeout (seconds)
timeout = 30

# Loop through each endpoint
for endpoint in endpoints:
    print(f"Running: {endpoint}")
    
    # Update params if the endpoint is 'data'
    if endpoint == "data":
        params = {"datasetid": "GHCND"}
        timeout = 60
    else:
        # Ensure 'datasetid' is not in params for other endpoints
        params = {"limit": 1000}

    # Define output filename
    filename = f"NCEI_{endpoint}.csv"
    
    # Set URL
    url_endpoint = f"{base_url}/{endpoint}"
    
    # Run a request using params and header dictionaries
    data = fetch_api_data(url_endpoint, params, headers, "GET", timeout)
    
    # Process and write the data to CSV
    if data and "results" in data:
        write_to_csv(data["results"], filename, "w")
    else:
        print(f"No data found or invalid response format for endpoint: {endpoint}")

Running: datasets
Running: datacategories
Running: datatypes
Running: locationcategories
Running: locations
Running: stations
Running: data
Http Error: 400 Client Error:  for url: https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND


AttributeError: 'tuple' object has no attribute 'tb_frame'